In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
%matplotlib inline
#import evolution
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

INFO: [2017-12-31 13:21:57,090 - rospy.topics] topicmanager initialized


INFO: [2017-12-31 13:21:57,233 - Configuration] Loading configuration file config.json
INFO: [2017-12-31 13:21:57,234 - Configuration] Using user specified environment: local
INFO: [2017-12-31 13:21:57,278 - VirtualCoach] Ready.


## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [2]:
def fitness_function(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
    fitness = 0
    for i in range(len(left_wheel)):
        if left_wheel[i] >= 0 and right_wheel[i] >= 0:
                   fitness += (left_wheel[i] + right_wheel[i])
    return fitness/float(2*len(left_wheel))

def get_top_performers(population, fitness_log, num_performers=15):
    """
    Extract the indices of the top individuals from the fitness log.
    
    :param fitness_log: fitness function scores for all individuals in a population
    :param num_performers: number for top performers to look for. Default value is
                           15, which corresponds to a truncation threshold of 25% in
                           this experiment.
    """
    top_performers = []
    for i in range(num_performers):
        max_index = np.argmax(fitness_log)
        print max_index
        top_performers.append(population[max_index])
        fitness_log[max_index] = -1

    return top_performers

def plot_trajectory(trajectory):
    plt.figure()
    plt.gca()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.ylim(-3, 3)
    plt.xlim(-3.9, 3.9)
    x_axis = [x[0] for x in trajectory[2:]]
    y_axis = [y[1] for y in trajectory[2:]]
    plt.plot([float(x) for x in x_axis], [float(y) for y in y_axis])

def plot_wheel_speeds(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[1:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[1:]])
    plt.plot(range(len(left_wheel)), left_wheel, 'b')
    plt.plot(range(len(right_wheel)), right_wheel, 'r')

Evolutionary Algorithm helper functions

In [3]:
def one_point_crossover(parent1, parent2):
        parent1 = parent1.reshape(290)
        parent2 = parent2.reshape(290)
        child1 = np.zeros(290, dtype=int)
        child2 = np.zeros(290, dtype=int)
        point = np.random.randint(len(parent1))
        for i in range(point):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        for i in range(point, 290):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        child1 = child1.reshape(10, 29)
        child2 = child2.reshape(10, 29)
        return child1, child2
    
def bit_mutation(population):
        for individual in population:
            individual = individual.reshape(290)
            for i in range(290):
                if np.random.rand() < 0.05:
                    individual[i] = 0 if individual[i] else 1
        return population
    
def get_unique_pairs(population):
    pairs = []
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            pairs.append((i, j))
    return pairs
            
def evolve_new_generation(top_performers):
    population = []
    for i in range(len(top_performers)):
        for j in range(4):
            population.append(top_performers[i])
    pairs = get_unique_pairs(population)
    for i in pairs:
        if np.random.rand() < 0.1:
            parent1 = population[i[0]]
            parent2 = population[i[1] - 1]
            child1, child2 = one_point_crossover(parent1, parent2)
            population[i[0]] = child1
            population[i[1]] = child2

    population = bit_mutation(population)
    rand = np.random.randint(len(population))
    population[rand] = top_performers[0]
    return population

## The Brain
The PyNN script that creates the neural network stored as a string. A new binary genetic string that encodes the connections between neurons is passed on each run.

In [4]:
brain = """from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)

l = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

receptors = []
for r in range(1,19):
    receptors.append(np.nonzero(l[:,r])[0])

def create_brain():

    dna = l

    NEURONPARAMS = {'v_rest': -60.5,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.4,
                    'v_reset': -60.5}

    SYNAPSE_PARAMS = {"weight": 1.0,
                      "delay": 2.0}

    population = sim.Population(10, sim.IF_cond_alpha())
    population[0:10].set(**NEURONPARAMS)


    # Connect neurons
    CIRCUIT = population

    SYN = sim.StaticSynapse(**SYNAPSE_PARAMS)

    row_counter=0
    for row in dna:
    	logger.info(row)
        n = np.array(row)
        r_type = 'excitatory'
        if n[0]==0:
            r_type = 'inhibitory'
        for i in range(19,29):
            if n[i]==1:
            	logger.info(str(18+row_counter)+' '+str(i-1)+' '+r_type)

                sim.Projection(presynaptic_population=CIRCUIT[row_counter:1+row_counter], postsynaptic_population=CIRCUIT[i-19:i-18], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        
        row_counter+=1

    sim.initialize(population, v=population.get('v_rest'))

    logger.debug("Circuit description: " + str(population.describe()))

    return population


circuit = create_brain()

"""

In [5]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

In [6]:
class FloreanoExperiment(object):
    
    def __init__(self, population, generations):
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')
        
    def on_status(self, status):
        self.last_status[0] = status
                 
    def save_simulation_data(self, trial):
        self.sim_data[trial].append([])
        wheel_speeds = self.sim.get_csv_data('wheel_speeds.csv')
        left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
        right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
        trajectory = self.sim.get_csv_data('robot_position.csv')
        fitness = fitness_function(wheel_speeds)
        self.sim_data[trial][-1] = {
            'fitness': fitness,
            'wheel_speeds': wheel_speeds,
            'left_wheel': left_wheel,
            'right_wheel': right_wheel,
            'trajectory': trajectory
        }

    def run_experiment(self):
        try:
            self.sim = vc.launch_experiment('floreano')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            self.sim_data.append([])
            for j in range(len(self.population)):
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(brain % genetic_string)
                self.sim.add_transfer_function(display_trial_tf % "Generation {}, Population {}".format(i, j) )
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(1000, lambda x: x['simulationTime'] == 40)
                self.sim.pause()
                self.save_simulation_data(i)
                self.sim.reset('full')
                self.wait_condition(100, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            self.fitness_log = [result['fitness'] for result in floreano_experiment.sim_data[i]]
            self.top_performers = get_top_performers(self.population, list(self.fitness_log))
            self.population = evolve_new_generation(self.top_performers)

In [7]:
population = np.random.randint(2, size=(60, 10, 29)) # random population of 30 binary genetic strings
floreano_experiment = FloreanoExperiment(population, 16)
floreano_experiment.run_experiment()

INFO: [2017-12-31 13:22:18,812 - VirtualCoach] Preparing to launch floreano.
INFO: [2017-12-31 13:22:18,813 - VirtualCoach] Retrieving list of experiments.
INFO: [2017-12-31 13:22:18,825 - Simulation] Attempting to launch floreano on localhost.
INFO: [2017-12-31 13:22:28,631 - Simulation (floreano - localhost #1)] Simulation Successfully Created.
INFO: [2017-12-31 13:22:28,641 - Simulation (floreano - localhost #1)] Ready.
INFO: [2017-12-31 13:22:28,642 - Simulation (floreano - localhost #1)] Status callback registered.
INFO: [2017-12-31 13:22:28,644 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:22:28,651 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-12-31 13:22:28,651 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:22:28,659 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:22:28,663 - Simulation (floreano - localhost #1)] Attempti

INFO: [2017-12-31 13:24:52,422 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 13:24:52,423 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:24:52,451 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:24:52,480 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:24:52,498 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:24:52,515 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 13:24:52,518 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 13:24:52,519 - Simulation (floreano - localhost #1)] Attempting to reset full
INFO: [2017-12-31 13:24:53,595 - Simulation (floreano - localhost #1)] [Resetting the simulation] 
INFO: [2017-12-31 13:24:53,596 - Simulation 

INFO: [2017-12-31 13:26:29,707 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-12-31 13:26:29,708 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:26:29,714 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:26:29,720 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:26:29,726 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 13:26:30,283 - Simulation (floreano - localhost #1)] Brain successfully updated.
INFO: [2017-12-31 13:26:30,284 - Simulation (floreano - localhost #1)] Attempting to retrieve transfer-function
INFO: [2017-12-31 13:26:30,293 - Simulation (floreano - localhost #1)] Attempting to set Transfer Function 
INFO: [2017-12-31 13:26:30,303 - Simulation (floreano - localhost #1)] Transfer Function '' successfully updated
INFO: [2017-12-31 13:26:30,304 - Simulation (floreano - localhost #1)] At

INFO: [2017-12-31 13:28:50,645 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 13:28:50,646 - Simulation (floreano - localhost #1)] Attempting to reset full
INFO: [2017-12-31 13:28:51,677 - Simulation (floreano - localhost #1)] [Resetting the simulation] 
INFO: [2017-12-31 13:28:51,678 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the 3D world
INFO: [2017-12-31 13:28:51,680 - Simulation (floreano - localhost #1)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2017-12-31 13:28:52,088 - Simulation (floreano - localhost #1)] [Resetting the simulation] Deleting: plane
INFO: [2017-12-31 13:28:52,495 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: pointlight_0
INFO: [2017-12-31 13:28:52,709 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-12-31 13:28:52,928 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
IN

INFO: [2017-12-31 13:30:31,118 - Simulation (floreano - localhost #1)] Attempting to set Transfer Function 
INFO: [2017-12-31 13:30:31,128 - Simulation (floreano - localhost #1)] Transfer Function '' successfully updated
INFO: [2017-12-31 13:30:31,129 - Simulation (floreano - localhost #1)] Attempting to transition to state: started
INFO: [2017-12-31 13:30:31,136 - Simulation (floreano - localhost #1)] Simulation state: started
INFO: [2017-12-31 13:31:15,680 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 13:31:15,685 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 13:31:15,685 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:31:15,723 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 13:31:15,746 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31

...

INFO: [2017-12-31 16:01:03,837 - Simulation (floreano - localhost #1)] Simulation state: started
INFO: [2017-12-31 16:01:47,389 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 16:01:47,395 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 16:01:47,395 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 16:01:47,413 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 16:01:47,430 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 16:01:47,451 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 16:01:47,470 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 16:01:47,475 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 16:01:47,47

INFO: [2017-12-31 16:03:35,386 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-12-31 16:03:35,638 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:03:35,645 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-12-31 16:03:35,647 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:03:35,654 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:03:35,659 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:03:35,665 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:03:36,194 - Simulation (floreano - localhost #1)] Brain successfully updated.
INFO: [2017-12-31 16:03:36,195 - Simulation (floreano - localhost #1)] Attempting to retrieve transfer-function
INFO: [2017-12-31 16:03:36,201 - Simu

INFO: [2017-12-31 16:06:09,777 - Simulation (floreano - localhost #1)] Attempting to retrieve recorders' csv data
INFO: [2017-12-31 16:06:09,800 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 16:06:09,805 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 16:06:09,806 - Simulation (floreano - localhost #1)] Attempting to reset full
INFO: [2017-12-31 16:06:10,900 - Simulation (floreano - localhost #1)] [Resetting the simulation] 
INFO: [2017-12-31 16:06:10,900 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the 3D world
INFO: [2017-12-31 16:06:10,908 - Simulation (floreano - localhost #1)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2017-12-31 16:06:11,315 - Simulation (floreano - localhost #1)] [Resetting the simulation] Deleting: plane
INFO: [2017-12-31 16:06:11,722 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: pointlight_0
INFO: 

INFO: [2017-12-31 16:07:57,523 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-12-31 16:07:58,053 - Simulation (floreano - localhost #1)] Brain successfully updated.
INFO: [2017-12-31 16:07:58,053 - Simulation (floreano - localhost #1)] Attempting to retrieve transfer-function
INFO: [2017-12-31 16:07:58,061 - Simulation (floreano - localhost #1)] Attempting to set Transfer Function 
INFO: [2017-12-31 16:07:58,070 - Simulation (floreano - localhost #1)] Transfer Function '' successfully updated
INFO: [2017-12-31 16:07:58,071 - Simulation (floreano - localhost #1)] Attempting to transition to state: started
INFO: [2017-12-31 16:07:58,076 - Simulation (floreano - localhost #1)] Simulation state: started
INFO: [2017-12-31 16:08:46,886 - Simulation (floreano - localhost #1)] Attempting to transition to state: paused
INFO: [2017-12-31 16:08:46,890 - Simulation (floreano - localhost #1)] Simulation state: paused
INFO: [2017-12-31 16:08:46,891 - Simulation (flo

ValueError: could not convert string to float: Reset)

In [ ]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][2:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][2:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)
